<a href="https://colab.research.google.com/github/edsrisk/portfolio_optimization/blob/main/dow_jones_optimal_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Portfolio Optimization: Creating an optimal portfolio of Dow Jones stocks for period 1/4/2021 to 7/23/2021

credit to: Computer Science youtube channel

In [41]:
# Import libraries and dependencies
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt

In [42]:
# Load NYSE data
from google.colab import files
files.upload()

{}

In [44]:
# read source file
dow_opt = pd.read_csv('dow_optimization.csv')
dow_opt.head()

,Date,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,WBA,WMT,VZ
0,1/4/2021,128.997803,223.416199,116.969841,202.720001,179.407867,220.309998,43.344891,82.519173,177.679993,52.908298,263.074677,260.908356,206.158539,120.954201,49.068108,154.565826,123.526443,51.933163,207.786865,169.220032,75.926445,216.689423,139.534210,135.272003,134.156097,346.991852,217.107101,40.658897,145.349930,56.966927
1,1/5/2021,130.592697,224.500443,117.594131,211.630005,180.875443,221.520004,43.364609,84.749947,178.440002,54.631439,268.961578,263.014008,206.664169,123.101204,49.996723,156.383087,124.198593,51.362255,209.032272,168.934494,76.057739,216.898438,140.659637,136.135727,133.048431,342.325562,213.866837,40.423191,144.576218,56.705570
2,1/6/2021,126.196747,229.911880,121.944321,211.029999,190.941589,216.149994,43.778736,87.477539,179.119995,57.206364,283.475342,264.516693,210.431412,126.175316,50.480778,157.854675,130.030441,49.728271,208.557831,171.504257,77.258156,211.274414,141.775116,137.568741,137.568100,356.691742,211.982513,42.259716,145.478897,57.392853
3,1/7/2021,130.502991,230.670868,120.979752,212.710007,191.306000,217.979996,44.330894,87.886673,178.580002,57.284687,289.531036,263.221649,210.778412,125.882545,51.557571,158.387985,134.300537,49.177044,209.526489,167.103180,78.796188,217.286652,144.364609,136.282959,137.538437,362.410400,213.168945,44.449795,145.468964,57.263702
4,1/8/2021,131.629379,235.076889,121.099075,209.899994,191.335571,222.039993,44.429497,88.724434,178.690002,56.990967,287.972443,266.019318,208.220627,125.433624,51.024120,158.062073,134.448822,50.279495,213.371475,164.050964,77.867737,218.610550,145.758957,136.224045,138.112045,360.782196,214.804016,44.400692,145.449127,56.549496


In [45]:
# Do a quick inspection of the file
dow_opt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    140 non-null    object 
 1   AAPL    140 non-null    float64
 2   AMGN    140 non-null    float64
 3   AXP     140 non-null    float64
 4   BA      140 non-null    float64
 5   CAT     140 non-null    float64
 6   CRM     140 non-null    float64
 7   CSCO    140 non-null    float64
 8   CVX     140 non-null    float64
 9   DIS     140 non-null    float64
 10  DOW     140 non-null    float64
 11  GS      140 non-null    float64
 12  HD      140 non-null    float64
 13  HON     140 non-null    float64
 14  IBM     140 non-null    float64
 15  INTC    140 non-null    float64
 16  JNJ     140 non-null    float64
 17  JPM     140 non-null    float64
 18  KO      140 non-null    float64
 19  MCD     140 non-null    float64
 20  MMM     140 non-null    float64
 21  MRK     140 non-null    float64
 22  MS

In [46]:
# Index on Data
dow_opt = dow_opt.set_index(pd.DatetimeIndex(dow_opt['Date'].values))
dow_opt.head()

,Date,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,WBA,WMT,VZ
2021-01-04,1/4/2021,128.997803,223.416199,116.969841,202.720001,179.407867,220.309998,43.344891,82.519173,177.679993,52.908298,263.074677,260.908356,206.158539,120.954201,49.068108,154.565826,123.526443,51.933163,207.786865,169.220032,75.926445,216.689423,139.534210,135.272003,134.156097,346.991852,217.107101,40.658897,145.349930,56.966927
2021-01-05,1/5/2021,130.592697,224.500443,117.594131,211.630005,180.875443,221.520004,43.364609,84.749947,178.440002,54.631439,268.961578,263.014008,206.664169,123.101204,49.996723,156.383087,124.198593,51.362255,209.032272,168.934494,76.057739,216.898438,140.659637,136.135727,133.048431,342.325562,213.866837,40.423191,144.576218,56.705570
2021-01-06,1/6/2021,126.196747,229.911880,121.944321,211.029999,190.941589,216.149994,43.778736,87.477539,179.119995,57.206364,283.475342,264.516693,210.431412,126.175316,50.480778,157.854675,130.030441,49.728271,208.557831,171.504257,77.258156,211.274414,141.775116,137.568741,137.568100,356.691742,211.982513,42.259716,145.478897,57.392853
2021-01-07,1/7/2021,130.502991,230.670868,120.979752,212.710007,191.306000,217.979996,44.330894,87.886673,178.580002,57.284687,289.531036,263.221649,210.778412,125.882545,51.557571,158.387985,134.300537,49.177044,209.526489,167.103180,78.796188,217.286652,144.364609,136.282959,137.538437,362.410400,213.168945,44.449795,145.468964,57.263702
2021-01-08,1/8/2021,131.629379,235.076889,121.099075,209.899994,191.335571,222.039993,44.429497,88.724434,178.690002,56.990967,287.972443,266.019318,208.220627,125.433624,51.024120,158.062073,134.448822,50.279495,213.371475,164.050964,77.867737,218.610550,145.758957,136.224045,138.112045,360.782196,214.804016,44.400692,145.449127,56.549496


In [47]:
dow_opt.info()
# Note the change in date format

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 140 entries, 2021-01-04 to 2021-07-23
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    140 non-null    object 
 1   AAPL    140 non-null    float64
 2   AMGN    140 non-null    float64
 3   AXP     140 non-null    float64
 4   BA      140 non-null    float64
 5   CAT     140 non-null    float64
 6   CRM     140 non-null    float64
 7   CSCO    140 non-null    float64
 8   CVX     140 non-null    float64
 9   DIS     140 non-null    float64
 10  DOW     140 non-null    float64
 11  GS      140 non-null    float64
 12  HD      140 non-null    float64
 13  HON     140 non-null    float64
 14  IBM     140 non-null    float64
 15  INTC    140 non-null    float64
 16  JNJ     140 non-null    float64
 17  JPM     140 non-null    float64
 18  KO      140 non-null    float64
 19  MCD     140 non-null    float64
 20  MMM     140 non-null    float64
 21  MRK     140 non-null

In [48]:
dow_opt.columns

Index(['Date', 'AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
       'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD',
       'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'WBA', 'WMT',
       'VZ'],
      dtype='object')

In [49]:
# Drop the date column
dow_opt.drop(columns=['Date'],axis=1,inplace=True)

In [50]:
dow_opt.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,WBA,WMT,VZ
2021-01-04,128.997803,223.416199,116.969841,202.720001,179.407867,220.309998,43.344891,82.519173,177.679993,52.908298,263.074677,260.908356,206.158539,120.954201,49.068108,154.565826,123.526443,51.933163,207.786865,169.220032,75.926445,216.689423,139.534210,135.272003,134.156097,346.991852,217.107101,40.658897,145.349930,56.966927
2021-01-05,130.592697,224.500443,117.594131,211.630005,180.875443,221.520004,43.364609,84.749947,178.440002,54.631439,268.961578,263.014008,206.664169,123.101204,49.996723,156.383087,124.198593,51.362255,209.032272,168.934494,76.057739,216.898438,140.659637,136.135727,133.048431,342.325562,213.866837,40.423191,144.576218,56.705570
2021-01-06,126.196747,229.911880,121.944321,211.029999,190.941589,216.149994,43.778736,87.477539,179.119995,57.206364,283.475342,264.516693,210.431412,126.175316,50.480778,157.854675,130.030441,49.728271,208.557831,171.504257,77.258156,211.274414,141.775116,137.568741,137.568100,356.691742,211.982513,42.259716,145.478897,57.392853
2021-01-07,130.502991,230.670868,120.979752,212.710007,191.306000,217.979996,44.330894,87.886673,178.580002,57.284687,289.531036,263.221649,210.778412,125.882545,51.557571,158.387985,134.300537,49.177044,209.526489,167.103180,78.796188,217.286652,144.364609,136.282959,137.538437,362.410400,213.168945,44.449795,145.468964,57.263702
2021-01-08,131.629379,235.076889,121.099075,209.899994,191.335571,222.039993,44.429497,88.724434,178.690002,56.990967,287.972443,266.019318,208.220627,125.433624,51.024120,158.062073,134.448822,50.279495,213.371475,164.050964,77.867737,218.610550,145.758957,136.224045,138.112045,360.782196,214.804016,44.400692,145.449127,56.549496


Portfolio Analysis: Current State

In [51]:
# Calculate daily returns
dow_opt_ret = dow_opt.pct_change()
dow_opt_ret.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,WBA,WMT,VZ
2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,0.012364,0.004853,0.005337,0.043952,0.008180,0.005492,0.000455,0.027033,0.004277,0.032568,0.022377,0.008070,0.002453,0.017751,0.018925,0.011757,0.005441,-0.010993,0.005994,-0.001687,0.001729,0.000965,0.008066,0.006385,-0.008257,-0.013448,-0.014925,-0.005797,-0.005323,-0.004588
2021-01-06,-0.033662,0.024104,0.036993,-0.002835,0.055652,-0.024242,0.009550,0.032184,0.003811,0.047133,0.053962,0.005713,0.018229,0.024972,0.009682,0.009410,0.046956,-0.031813,-0.002270,0.015212,0.015783,-0.025929,0.007930,0.010526,0.033970,0.041966,-0.008811,0.045432,0.006244,0.012120
2021-01-07,0.034123,0.003301,-0.007910,0.007961,0.001908,0.008466,0.012612,0.004677,-0.003015,0.001369,0.021362,-0.004896,0.001649,-0.002320,0.021331,0.003378,0.032839,-0.011085,0.004645,-0.025662,0.019908,0.028457,0.018265,-0.009346,-0.000216,0.016032,0.005597,0.051824,-0.000068,-0.002250
2021-01-08,0.008631,0.019101,0.000986,-0.013211,0.000155,0.018626,0.002224,0.009532,0.000616,-0.005127,-0.005383,0.010629,-0.012135,-0.003566,-0.010347,-0.002058,0.001104,0.022418,0.018351,-0.018265,-0.011783,0.006093,0.009659,-0.000432,0.004171,-0.004493,0.007670,-0.001105,-0.000136,-0.012472


In [52]:
# Calculate the annualized covariance of returns across all 30 dow stocks
dow_opt_cov = dow_opt_ret.cov()*252
dow_opt_cov

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,WBA,WMT,VZ
AAPL,0.073677,0.013981,0.002776,0.020996,-0.005445,0.035257,0.021290,0.000904,0.004293,0.007850,0.003171,0.017033,0.010945,0.000591,0.039006,0.006483,0.000038,0.010740,0.014671,0.005742,-0.001920,0.041884,0.019101,0.003969,0.009075,0.002244,0.023283,0.004019,0.014040,-0.002546
AMGN,0.013981,0.039333,0.000941,0.003895,0.006989,0.004365,0.013673,0.005659,0.005482,0.009607,0.000664,0.012471,0.006976,0.009278,0.012030,0.014212,0.002652,0.005918,0.010658,0.005277,0.018015,0.013368,0.011110,0.010609,0.011366,0.011935,0.005825,0.010603,0.009843,0.008429
AXP,0.002776,0.000941,0.073079,0.049892,0.043835,0.006093,0.013930,0.046608,0.040374,0.052985,0.043679,0.009741,0.034296,0.017445,0.027827,0.004499,0.038398,0.009815,0.010987,0.013682,0.002514,0.000565,0.018704,0.002966,0.036536,0.013284,0.025426,0.008663,0.005663,0.010125
BA,0.020996,0.003895,0.049892,0.136168,0.046555,0.015770,0.014313,0.044860,0.036490,0.053945,0.054157,0.011893,0.040358,0.020701,0.042438,0.005184,0.040944,0.005412,0.017611,0.007084,-0.002357,0.013474,0.014843,0.000620,0.029489,0.006121,0.019216,0.025226,0.018325,0.004702
CAT,-0.005445,0.006989,0.043835,0.046555,0.067953,-0.001082,0.010474,0.038511,0.029515,0.055826,0.046260,0.008123,0.029212,0.017188,0.023094,0.005728,0.040168,0.004175,0.009122,0.010761,0.008662,-0.006970,0.007887,0.004356,0.031298,0.012883,0.010256,0.015737,0.008367,0.009271
CRM,0.035257,0.004365,0.006093,0.015770,-0.001082,0.066776,0.013783,0.005329,0.007438,0.005007,0.001513,0.004755,0.007080,-0.003346,0.031060,0.004986,0.000982,0.008697,0.012068,-0.000826,0.000418,0.032509,0.015832,0.004766,-0.002495,-0.001170,0.020407,-0.005342,0.010721,0.000024
CSCO,0.021290,0.013673,0.013930,0.014313,0.010474,0.013783,0.034473,0.014028,0.014893,0.018009,0.012021,0.016551,0.016594,0.017717,0.022069,0.011903,0.012578,0.013103,0.011995,0.016217,0.007986,0.017370,0.020160,0.010919,0.016385,0.008035,0.019851,0.016363,0.011023,0.007329
CVX,0.000904,0.005659,0.046608,0.044860,0.038511,0.005329,0.014028,0.073717,0.022336,0.058073,0.040663,0.002865,0.025361,0.021380,0.020502,0.011207,0.034090,0.008783,0.006130,0.013820,0.008734,0.002201,0.011054,0.004642,0.029153,0.011740,0.015360,0.017432,0.007809,0.013288
DIS,0.004293,0.005482,0.040374,0.036490,0.029515,0.007438,0.014893,0.022336,0.065509,0.030940,0.031226,0.013438,0.023016,0.006480,0.016698,0.005821,0.024942,0.013473,0.012904,0.004066,0.004262,0.006728,0.017739,0.007680,0.023385,0.010166,0.025241,0.006185,0.008110,0.010189
DOW,0.007850,0.009607,0.052985,0.053945,0.055826,0.005007,0.018009,0.058073,0.030940,0.097040,0.057407,0.014300,0.036208,0.024208,0.024878,0.010075,0.046444,0.010049,0.009878,0.016748,0.009169,0.001475,0.019696,0.005122,0.036730,0.012136,0.018954,0.021893,0.015087,0.012989


In [68]:
# Set weights and confirm length, i.e. number of stocks
weights = np.array([0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333,	0.0333333])
print(len(weights))
weights



30


array([0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333,
       0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333,
       0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333,
       0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333,
       0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333])

In [69]:
sum = 0
for weight in weights:
  sum += weight
print(sum)

0.9999990000000002


In [72]:
# Calculate the portfolio variance
dow_opt_var = np.dot(weights.T, np.dot(dow_opt_cov, weights))
print('Portfolio Variance: ', dow_opt_var)

Portfolio Variance:  0.01514443899809616


In [73]:
# Calculate portfolio volatility
dow_opt_vol = np.sqrt(dow_opt_var)
print('Portfolio Volatility / Standard Deviation: ', dow_opt_vol)

Portfolio Volatility / Standard Deviation:  0.12306274415149436


In [74]:
# Calculate weighted average returns, annualized
dow_opt_port_ret = np.sum(dow_opt_ret.mean()*weights)*252
print('Portfolio Annualized Returns: ', dow_opt_port_ret)

Portfolio Annualized Returns:  0.28422688681390035


In [75]:
# Print Portfolio Summary: Current State
print(" Portfolio Summary")
print("----------------------------")
#print("Assets: ", portfolio_df[:0])
print('Annual Return: ', round(dow_opt_port_ret, 2) * 100, '% / Volatility: ', round(dow_opt_vol, 2) * 100, '% / Variance: ', round(dow_opt_var, 2) * 100, '%')

 Portfolio Summary
----------------------------
Annual Return:  28.000000000000004 % / Volatility:  12.0 % / Variance:  2.0 %


In [77]:
# Portfolio Optimization
# Install portfolio optimization package
!pip install PyPortfolioOpt

     |████████████████████████████████| 60 kB 3.1 MB/s 
     |████████████████████████████████| 2.7 MB 9.0 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [81]:
# import libraries and dependencies
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [82]:
# Calculate expected annualized returns and sample covariance matrix
mu = expected_returns.mean_historical_return(dow_opt)
S = risk_models.sample_cov(dow_opt)

In [83]:
# Apply modern portfolio theory: max sharpe ratio, efficient ratio, allocate weithts for optimal allocation
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.0), ('AMGN', 0.0), ('AXP', 0.22879), ('BA', 0.0), ('CAT', 0.0), ('CRM', 0.0), ('CSCO', 0.0), ('CVX', 0.0), ('DIS', 0.0), ('DOW', 0.0), ('GS', 0.19552), ('HD', 0.09627), ('HON', 0.0), ('IBM', 0.0), ('INTC', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('KO', 0.0), ('MCD', 0.0), ('MMM', 0.05392), ('MRK', 0.0), ('MSFT', 0.38287), ('NKE', 0.0), ('PG', 0.0), ('TRV', 0.0), ('UNH', 0.04263), ('V', 0.0), ('WBA', 0.0), ('WMT', 0.0), ('VZ', 0.0)])
Expected annual return: 76.7%
Annual volatility: 14.3%
Sharpe Ratio: 5.23


(0.7668347213661998, 0.14267621564669028, 5.234472459064865)

In [87]:
# Stock Returns
mu

AAPL    0.291723
AMGN    0.205881
AXP     1.036900
BA      0.174433
CAT     0.324956
CRM     0.241683
CSCO    0.551635
CVX     0.387569
DIS    -0.015658
DOW     0.260315
GS      0.892825
HD      0.554945
HON     0.212919
IBM     0.326280
INTC    0.149983
JNJ     0.211117
JPM     0.432982
KO      0.184229
MCD     0.327791
MMM     0.359901
MRK     0.038860
MSFT    0.692587
NKE     0.375472
PG      0.061372
TRV     0.192979
UNH     0.399666
V       0.282271
WBA     0.277021
WMT    -0.036123
VZ     -0.034322
dtype: float64

In [89]:
!pip install pulp

     |████████████████████████████████| 40.6 MB 32 kB/s 


In [92]:
# Portfolio Rebalancing
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
dow_opt_value = 100000  # variable to hold the amount of money you would like to invest in the optimal portfolio
latest_prices = get_latest_prices(dow_opt)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = dow_opt_value)
allocation, leftover = da.lp_portfolio()
print('Discrete Allocation: ', allocation)
print('Funds Remaining: $', leftover)

Discrete Allocation:  {'AXP': 133, 'GS': 52, 'HD': 29, 'MMM': 27, 'MSFT': 132, 'UNH': 10}
Funds Remaining: $ 37.42970000000787


Optimal Portfolio for an investment of $100,000
133 shares American Express,
52 shares of Goldman Sachs,
29 shares of Home Depot,
27 shares of 3M,
132 shares of Microsoft,
10 shares of United Healthcare.

Unspent funds: $37.43